In [131]:
import os
import requests
from bs4 import BeautifulSoup

In [132]:
seznam_url = "https://gcatholic.org/documents/data/type-PEN.htm"

In [133]:
seznam_stranka = requests.get(seznam_url)

In [134]:
seznam_polivka = BeautifulSoup(seznam_stranka.text)

In [159]:
dokumenty = []
for radek in seznam_polivka.find_all('tr'):
    if 'Pope' in radek.text:
        dokument = {}
        dokument['nazev'] = radek.find(class_='doc').text.strip()
        dokument['dokument'] = 'encyklika'
        tagy = []
        for t in radek.find_all(class_='tag'):
            tagy.append(t.text.strip())
        dokument['tagy'] = tagy
        dokument['papez'] = radek.find_all('td')[1].text.strip()
        datum = radek.find_all('td')[2].text.strip()
        if datum == '1831.03':
            datum = '1831.03.01'
        dokument['datum'] = datum
        dokumenty.append(dokument)

In [161]:
dokumenty

[{'nazev': 'Dilexit nos',
  'dokument': 'encyklika',
  'tagy': ['Sacred Heart'],
  'papez': 'Pope Francis',
  'datum': '2024.10.24'},
 {'nazev': 'Fratelli tutti',
  'dokument': 'encyklika',
  'tagy': ['inter-religious dialogue', 'social'],
  'papez': 'Pope Francis',
  'datum': '2020.10.03'},
 {'nazev': 'Laudato si’',
  'dokument': 'encyklika',
  'tagy': ['ecology', 'social'],
  'papez': 'Pope Francis',
  'datum': '2015.05.24'},
 {'nazev': 'Lumen Fidei',
  'dokument': 'encyklika',
  'tagy': ['faith'],
  'papez': 'Pope Francis',
  'datum': '2013.06.29'},
 {'nazev': 'Caritas in Veritate',
  'dokument': 'encyklika',
  'tagy': ['social'],
  'papez': 'Pope Benedict XVI',
  'datum': '2009.06.29'},
 {'nazev': 'Spe Salvi',
  'dokument': 'encyklika',
  'tagy': ['hope'],
  'papez': 'Pope Benedict XVI',
  'datum': '2007.11.30'},
 {'nazev': 'Deus caritas est',
  'dokument': 'encyklika',
  'tagy': ['love'],
  'papez': 'Pope Benedict XVI',
  'datum': '2005.12.25'},
 {'nazev': 'Ecclesia de Eucharistia

In [163]:
import polars as pl

In [165]:
df = pl.DataFrame(dokumenty).with_columns(pl.col('datum').str.to_date())

In [167]:
df

nazev,dokument,tagy,papez,datum
str,str,list[str],str,date
"""Dilexit nos""","""encyklika""","[""Sacred Heart""]","""Pope Francis""",2024-10-24
"""Fratelli tutti""","""encyklika""","[""inter-religious dialogue"", ""social""]","""Pope Francis""",2020-10-03
"""Laudato si’""","""encyklika""","[""ecology"", ""social""]","""Pope Francis""",2015-05-24
"""Lumen Fidei""","""encyklika""","[""faith""]","""Pope Francis""",2013-06-29
"""Caritas in Veritate""","""encyklika""","[""social""]","""Pope Benedict XVI""",2009-06-29
…,…,…,…,…
"""Magnæ nobis""","""encyklika""","[""marriage"", ""Poland""]","""Pope Benedict XIV""",1748-06-29
"""Vix pervenit""","""encyklika""","[""Italy""]","""Pope Benedict XIV""",1745-11-01
"""Nimiam licentiam""","""encyklika""","[""marriage"", ""Poland""]","""Pope Benedict XIV""",1743-05-18


In [169]:
df.write_parquet('data/encykliky.parquet', use_pyarrow=True)